In [4]:
from random import sample
from scipy import stats
from scipy.stats import chisquare
import random
from collections import defaultdict
import numpy as np
import math

# Pval simulation

## Generation of random counts

The following function generates a list of `n` random numbers selected from the range `[0, 1, 2, 3]` and converts them to a list of four counts (number of occurences of `0`, `1`, `2`, `3`).

In [5]:
def generate_random_counts(n:int):
    list1 = [0, 1, 2, 3]
    sampled_list = []
    for _ in range(n):
        rval = random.sample([0, 1, 2, 3],1)
        sampled_list.append(rval[0])
    d = defaultdict(int)
    d[0] = 0; d[1] = 0; d[2] = 0; d[3] = 0;
    for x in sampled_list:
        d[x] += 1
    random_counts = []
    for k,v in d.items():
        random_counts.append(v)
    return random_counts

In [18]:
random_counts = generate_random_counts(10)
random_counts

[3, 2, 5, 0]

## Simple/Twisted binomial P-value

The following function takes a list of four random counts, sums the counts for `0` and `1` and calculates a binomial P-value.

In [19]:
def binom12(random_counts, verbose=False):
    N = sum(random_counts)
    K = random_counts[0] + random_counts[1]
    pval = stats.binom_test(K, n=N, p=0.5, alternative='two-sided')
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts))
        print('Number of successes (K): ' + str(K))
        print('Number of trials (N): ' + str(N))
    return pval

In [20]:
pval = binom12(generate_random_counts(1000), verbose=True)
print('Binomial-01 P-value: ' + str(pval))

Randomly generated list of four counts: [243, 226, 277, 254]
Number of successes (K): 469
Number of trials (N): 1000
Binomial-01 P-value: 0.053677849645009976


## Highest two counts binomial P-value

The function above takes a list of four random counts, sums the highest two counts, and calculates a binomial P-value.

In [21]:
def binomHT(random_counts, verbose=False):
    N = sum(random_counts)
    sorted_counts = sorted(random_counts)
    K = sum(sorted_counts[2:])
    pval = stats.binom_test(K, n=N, p=0.5, alternative='greater')
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts)) 
        print('Sorted four counts: ' + str(sorted_counts))
        print('Number of successes (K): ' + str(K))
        print('Number of trials (N): ' + str(N))   
    return pval

In [22]:
pval = binomHT(generate_random_counts(1000), verbose=True)
print('Binomial-HT P-value: ' + str(pval))

Randomly generated list of four counts: [259, 251, 255, 235]
Sorted four counts: [235, 251, 255, 259]
Number of successes (K): 514
Number of trials (N): 1000
Binomial-HT P-value: 0.19660913023371587


## Chi-squared P-value

The following function takes a list of four random counts and calculates a chi-squared P-value.

In [23]:
def chisq_pval(random_counts, verbose=False):
    pval = stats.chisquare(f_obs=random_counts)[1]
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts))     
    return pval

In [24]:
pval = chisq_pval(generate_random_counts(1000), verbose=True)
print('Chi-square P-value: ' + str(pval))

Randomly generated list of four counts: [247, 250, 250, 253]
Chi-square P-value: 0.9949712955863822


## Barnard exact P-value

The [documementation for the chi-square](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) test states that the test is not valid if the counts are too small:

*This test is invalid when the observed or expected frequencies in each category are too small. A typical rule is that all of the observed and expected frequencies should be at least 5. According to [3], the total number of samples is recommended to be greater than 13, otherwise exact tests (such as Barnard’s Exact test) should be used because they do not overreject.*


The following function takes a list of four random counts, converts them into a contingency table and calculates a Barnard exact P-value.

In [25]:
def barnard_exact_pval(random_counts, verbose=False):       
    contingency_table = [[random_counts[0], random_counts[1]],[random_counts[2], random_counts[3]]]
    res = stats.barnard_exact(contingency_table)
    if verbose:
        print('Randomly generated list of four counts: ' + str(random_counts))
        print('Contingency table: ' + str(contingency_table))    
    return res.pvalue

In [26]:
pval = barnard_exact_pval(generate_random_counts(1000), verbose=False)
print('Barnard exact P-value: ' + str(pval))

Barnard exact P-value: 0.32060072842436416


## Comparison of different P-values

We have a function that randomly generates a list of four random counts and four different ways of computing a P-value. The following code generates a certain number of random count lists and, for each list, all four P-values are calculated and added to lists. At the end, the number of P-values are determined that are below a specified P-value threshold. Because the lists were randomly generated, a certain number of tests can be expected to be signifcant. This number depends on the number of iterations and the P-value threshold.

In [38]:
simplePvals = []
htPvals = []
chi2_pvals = []
barnard_pvals = []

samplesize = 11
iter_num = 10000
pval_thresh = 0.05

print("Expected number of times 'significant': " + str(iter_num*pval_thresh))


for _ in range(iter_num):
    
    random_counts = generate_random_counts(samplesize)
    
    pval = binom12(random_counts)
    simplePvals.append(pval)
    
    pvalHT = binomHT(random_counts)
    htPvals.append(pvalHT)
    
    pvalChi2 = chisq_pval(random_counts)
    chi2_pvals.append(pvalChi2)
    
    pvalBarnard = barnard_exact_pval(random_counts)
    barnard_pvals.append(pvalBarnard)

print("Simple pval times 'significant'", sum([x<pval_thresh for x in simplePvals]))
print("HT pval  times 'significant'", sum ([x<pval_thresh for x in htPvals]))
print("Chi-square pval times 'significant'",sum ([x<pval_thresh for x in chi2_pvals]))
print("Barnard exact pval times 'significant'",sum ([x<pval_thresh for x in barnard_pvals]))

Expected number of times 'significant': 500.0
Simple pval times 'significant' 104
HT pval  times 'significant' 1808
Chi-square pval times 'significant' 453
Barnard exact pval times 'significant' 359


## Precision of the Barnard exact P-value

In [92]:
for i in range(10,1541,10):
    increasing_counts = [i, 0, 0, i]
    print(increasing_counts)
    pval = barnard_exact_pval(increasing_counts)
    print(pval)
    if sys.float_info.min * sys.float_info.epsilon < pval:
        print(-math.log10(pval))
    else:
        print('INF')
        


[10, 0, 0, 10]
1.9073486328125e-06
5.719569917615643
[20, 0, 0, 20]
1.81898940354586e-12
11.740169830895265
[30, 0, 0, 30]
1.7347234759768136e-18
17.76076974417489
[40, 0, 0, 40]
1.6543612251060704e-24
23.781369657454512
[50, 0, 0, 50]
1.5777218104420464e-30
29.801969570734133
[60, 0, 0, 60]
1.5046327690525364e-36
35.82256948401376
[70, 0, 0, 70]
1.4349296274686487e-42
41.84316939729337
[80, 0, 0, 80]
1.3684555315672264e-48
47.863769310573005
[90, 0, 0, 90]
1.305060893599733e-54
53.88436922385262
[100, 0, 0, 100]
1.2446030555722618e-60
59.90496913713225
[110, 0, 0, 110]
1.1869459682199626e-66
65.92556905041188
[120, 0, 0, 120]
1.1319598848533494e-72
71.9461689636915
[130, 0, 0, 130]
1.0795210693868367e-78
77.96676887697112
[140, 0, 0, 140]
1.029511517893597e-84
83.98736879025076
[150, 0, 0, 150]
9.818186930595562e-91
90.00796870353037
[160, 0, 0, 160]
9.363352709384684e-97
96.02856861680999
[170, 0, 0, 170]
8.929588994393221e-103
102.0491685300896
[180, 0, 0, 180]
8.515919680016169e-10

There is a limit to the precision that we must must consider in our code.

In [39]:
import sys
-math.log10(sys.float_info.min * sys.float_info.epsilon)

323.3062153431158